In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import argparse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, Input
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, LeakyReLU, LSTM
from keras.losses import mse, binary_crossentropy
from keras.utils import np_utils, plot_model
from keras import backend as K
from keras import optimizers
from keras import regularizers
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/audio-binary-classification/train_data.npy
/kaggle/input/audio-binary-classification/train_labels.csv
/kaggle/input/audio-binary-classification/sample_submission.csv
/kaggle/input/audio-binary-classification/test_data.npy


In [2]:
train_data = np.load('/kaggle/input/audio-binary-classification/train_data.npy');
test_data = np.load('/kaggle/input/audio-binary-classification/test_data.npy');


print("inputs loaded")

inputs loaded


In [3]:
#Scale and Reshape
train_data = train_data.astype('float32');
test_data = test_data.astype('float32');

#Normalize Data
#train_data /= np.amax(test_data);
#test_data /= np.amax(test_data);

print(train_data.shape);

train_data = train_data.reshape(-1,44100,1);
test_data = test_data.reshape(-1,44100,1);

input_shape = train_data.shape;
print(input_shape);

(3999, 44100)
(3999, 44100, 1)


In [4]:
#Look at CSV and store as array
train_labels = pd.read_csv("/kaggle/input/audio-binary-classification/train_labels.csv");
train_labels = train_labels.values;
train_labels = train_labels[:,1];
print(train_labels[:10]);
print(train_labels.shape)

[0 0 0 1 0 0 1 0 1 0]
(3999,)


In [5]:
# create convolution neural network
model = Sequential()
model.add(LSTM(16, input_shape=(44100,1),return_sequences=True))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.2))

#model.add(MaxPooling1D(pool_size=128))

model.add(Flatten())

model.add(Dense(16))
model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.33))

model.add(Dense(1, activation='sigmoid'))

model.summary();

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 44100, 16)         1152      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 44100, 16)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 44100, 16)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 705600)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                11289616  
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
__________

In [6]:
#Compile model
sgd = optimizers.SGD(lr=.01);
model.compile(loss='MSE', 
              optimizer='sgd',
              metrics=['accuracy'])

In [7]:
model.fit(train_data,train_labels,
          batch_size=30,epochs=4,verbose=1)

Epoch 1/4
3999/3999 [==============================] - 4171s 1s/step - loss: 0.1311 - acc: 0.8712
Epoch 2/4
3999/3999 [==============================] - 4163s 1s/step - loss: 0.1126 - acc: 0.8780
Epoch 3/4
3999/3999 [==============================] - 4237s 1s/step - loss: 0.1103 - acc: 0.8780
Epoch 4/4
3999/3999 [==============================] - 4256s 1s/step - loss: 0.1116 - acc: 0.8780


In [8]:
model.save("lstm_failure.h5")
Y_pred = model.predict(test_data);
print(Y_pred)
df_pred = pd.DataFrame(Y_pred);
df_pred.index.name = "Id"
df_pred = pd.DataFrame.rename(df_pred,columns={0:"Label"})
print(df_pred);
df_pred.to_csv("output.csv")

[[0.12664011]
 [0.12780091]
 [0.12804177]
 ...
 [0.12678288]
 [0.12736703]
 [0.12791583]]
         Label
Id            
0     0.126640
1     0.127801
2     0.128042
3     0.126385
4     0.126788
...        ...
3992  0.128003
3993  0.126775
3994  0.126783
3995  0.127367
3996  0.127916

[3997 rows x 1 columns]
